 #  Byte degree 미니 프로젝트
 ## Daum 영화 제목 및 댓글 크롤링

### 문제
    - https://movie.daum.net 에 접속하여 본인이 좋아하는 영화를 선택합니다.
    (1) 메인 Tab에서 영화 제목을 크롤링 합니다.
    (2) 평점 Tab에서 영화 댓글을 크롤링 합니다.

### 조건 
    (1) 각 크롤링은 함수을 만들어 호출하는 형태로 작성한합니다. (영화 제목과 댓글 총 2개의 함수가 만들어져야 함)
    (2) 영화 제목 크롤링 함수는 인자값으로 영화 ID를, 댓글 크롤링 함수는 인자값으로 영화 ID와 최대 page 수를 받을 수 있어야 합니다. 
    (3) 댓글 크롤링 후 최종 결과는 댓글의 Value 만 크롤링 되어야 하며 불필요한 HTML 테그가 결과에 포함되어서는 안됩니다.
    (4) 만약 댓글의 전체 page 수가 함수에 인자로 지정한 최대 page 수 보다 적게 존재하는 경우는 이를 체크하여 크롤링을 멈춥니다. 
        (e.g. 함수 인자에 최대 크롤링 page를 100으로 지정했다 하더라도 실제 크롤링 대상 page가 10개만 존재한다면 크롤링 멈춤)
    (5) 모든 작성은 jupyter notebook에서 하며 수행 결과를 포함시켜 ipynb 확장자 파일로 제출합니다.

### 추가 hint
    (1) 영화의 댓글은 get 방식으로 page 별로 호출 할 수 있습니다. (page는 1부터 시작)
    (2) BeautifulSoap의 find_all을 활용하면 리스트로 반환된 결과를 len 함수를 통해 개수를 확인할 수 있습니다.

In [1]:
import requests
from bs4 import BeautifulSoup

### 영화 메인 정보 크롤링 함수 (메인 Tab)

In [2]:
def get_daum_movie_title(movie_id):   # 체크 : movie_id를 받을 수 있어야 함
    url = 'https://movie.daum.net/moviedb/main?movieId={}'.format(movie_id)
    resp = requests.get(url)
    
    soup = BeautifulSoup(resp.text)
    
    # 영화 제목만 크롤링
    title_tag = soup.select_one('strong.tit_movie')
    if title_tag:
        return title_tag.get_text()
    return ""

In [3]:
# 영화 메인 제목 크롤링 함수 호출
# '다크 나이트 (2008)' movieId=43569
# 크롤링 결과 확인
get_daum_movie_title('43569')

'다크 나이트 (2008)'

In [4]:
# 영화 메인 제목 크롤링 함수 호출
# '어쩌다 아스널 (2019)' movieId=134377
# 크롤링 결과 확인
get_daum_movie_title('134377')

'어쩌다 아스널 (2019)'

---
>- 리뷰
   - 잘해주셨습니다
   - url 분석을 잘해주셔ㅕ네요
   - 제목이 없을 경우, movieId 가 없을 경우에 대한 에러 처리도 잘해주셨어요!

### 영화 댓글 크롤링 함수 (평점 Tab)

In [5]:
## 영화 댓글 클롤링 함수
# 인자값으로 영화 ID와 최대 page 수
def get_daum_movie_comments(movie_id, max_page):
    # header 데이터 구성하기
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36'
    }
    
    url_template = 'https://movie.daum.net/moviedb/grade?movieId={}&type=netizen&page={}'
    
    # 댓글 page 1 부터 시작
    page = 1
    review = ''
    while True:
        # header 데이터 함께 전달
        url = url_template.format(movie_id, page)
        resp = requests.get(url, headers=headers)
        
        # status_codeps_code 에러 확인하기
        if resp.status_code == 200:
            soup = BeautifulSoup(resp.text)
            
            # 현재 page의 댓글의 갯수가 0 이면 크롤링 멈춤 
            if len(soup.find_all('p', class_='desc_review')) == 0:
                break
            
            # 댓글의 Value 만 크롤링
            for p in soup.find_all('p', class_='desc_review'):
                review += p.get_text().strip()
            
            # 다음 page 크롤링 준비
            page += 1
            
            # 인자로 지정한 최대 page 를 넘어가면 크롤링을 멈춤
            if page > max_page:
                break
        else:
            print('error')
            break
    
    return review

In [6]:
## 크롤링 결과 확인
# https://movie.daum.net/moviedb/grade?movieId=43569
get_daum_movie_comments('43569', 5)

'배트맨보다 더 매력넘치는 조커진정 악에 맞서는 정의가 되고자 한다면 어둠조차 받아들여라영화정의란 무엇인가. \r인간은 자신의 이익에 따라 두가지 가면을 쓰고 있다.배트맨과 조커!\rDC영화중에서 가장 재미있고 명작이다!\rWhy so serious?\r호랑이는 가죽을 남겼고, 히스 레저는 조커를 남겼다!히어로물에 연기력과 철학까지 담아낼 줄이야. 조커 연기는 다시 봐도 압권.베트맨과 악당일뿐디씨의 다크함이 잘 드러난 영화이자 디씨의 마지막 호황기와...이제 봐도 goodgood10년이 넘은 영화지만 너무나 잘만들었고\r조커의 연기는 너무나 완벽했다\r배트맨.슈퍼맨 시리즈는 좋아하지 않지만\r이 다크나이트 만큼은 정말 재미있게 보았다\r너무나 유명한 영화라서 더이상 설명은 \r무의미한 영화...조커가 히스레저 그사람이란걸\r뒤늦게 알았음.\r히스레저 \r당신은 훌륭한 배우였어요그냥 말이필요없다캐릭터들이 주는 몰입감.혼돈은 모두에게 공평하다.크리스토퍼 놀란의 배트맨이 아닌 다른 감독의 배트맨은 어떨지 상상도 안되지만 진정한 슈퍼히어로는 진짜 크리스찬 베일의 배트맨!!!11년이 지나도 최고의 슈퍼히어로 영화는 여전히 다크나이트다조커 보고 .. 다시 보니 \r지금의 개검과 자한당 기레기가 자꾸 떠오른다.명작질리도록 봐도 질리지 않는 영화!다시 봐도 역시 히스 레저.조커 를 보고 생각나서 놀란 배트맨 3부작 다시 정주행. 다시 봐도 다크나이트는 짱.조커를 보고\r다시 한번 다크나이트를 봤다.\r아까운 배우. 히.스.레.져.히스레저라는 진짜 조커를 탄생기킨 시대의 명작 \r배트맨 영화 중 최고다 \r배트맨보다 조커가 더 강렬했던정의란 무엇인가?이건 뭐.. 다시봐도 레전드..연기를 잘 하는 수준을 넘어선 히스 레져.\r걸음걸이, 시선처리, 숨 쉬는 것. 하다 못해 머리카락 한올 까지도 조커 그 자체 였다. 너무 아쉽다. 그런 배우가 더 이상 이 세상에 없다는게.이영화의  깊이를  마블은 따라갈수가 없지10년만에 다시봐도 걸작. 배트맨이라는 캐릭터를 가진 미국 헐리우드가 

In [7]:
## 크롤링 결과 확인
# https://movie.daum.net/moviedb/grade?movieId=134377
# page 10000 가기전에 크롤링 멈춤 확인
get_daum_movie_comments('134377', 10000)


'4스날4.4가즈아432스날4스날어째서 아스날이젠 사스날도 아니다 10스날!!!!!정말 재미밌습니다. 아스날 화이팅4444ㄱㄱ4스날 4점개의,  개를 위한,  개에 의한 영화... 왈왈!!와우!사스날!손지환 4스날4스날4스날 가즈아~황스날 가즈아~아스날이 토트넘 보다는 낫지개집이여도 닭집보다 근본있음\r닭집이랑 비교해서 미안하긴하지만\r지금은 닭집이 더 잘함\r그래도 무근본이지반 페르시왈! 왈!'

---
>- 리뷰
   - 아주 잘해주셨습니다!
   - break 조건도 완벽하게 걸어주셨네요!
   - 참고로 리뷰나 각 하나마다 의미가 있는 경우
   - 위와 같이 string으로 붙여서 모은 것보다
   - 한줄한줄이 독립적인 list 형태로 모으는 것이 더 좋습니다.